In [ ]:
#| default_exp diff_expr

# Volcano and MA Plot Visualization for Omics Data

> This notebook provides interactive visualization tools for displaying omics data in two common formats: Volcano plots and MA plots. 


In [ ]:
#| hide
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Data Loading and Processing

> The PlotData class handles loading data from CSV files and mapping columns to standard names. 

In [ ]:
#| export
class PlotData:
    def __init__(self, file_path, column_mapping=None, log_fdr=False, highlight_ids=None):
        """
        Initialize the PlotData class for various omics experiments.
        
        Parameters:
        -----------
        file_path : str
            Path to the CSV file containing omics data
        column_mapping : dict, optional
            Dictionary mapping standard column names to the actual column names in the file
            Default column names expected: 'log2fc', 'fdr', 'avg_intensity', 'id', 'description'
        log_fdr : bool, optional
            Whether to calculate -log10 of FDR values (default: False)
        highlight_ids : list, optional
            List of IDs to highlight in the plots (default: None)
        """
        self.file_path = file_path
        self.column_mapping = column_mapping or {
            'log2fc': 'logFC',
            'fdr': 'adj.P.Val',
            'avg_intensity': 'AveExpr',
            'id': None,  # Will use index if None
            'description': None  # May not be present in all datasets
        }
        self.log_fdr = log_fdr
        self.highlight_ids = highlight_ids or []
        self.highlight_indices = []  # Will be populated after loading data
        self.data = None
        self.load_data()
        
    def load_data(self):
        """Load and process the data from the CSV file."""
        try:
            self.data = pd.read_csv(self.file_path, index_col=0)
            
            # Map the columns and store as class attributes
            for std_col, file_col in self.column_mapping.items():
                if file_col is None and std_col == 'id':
                    # Use index as ID if not specified
                    setattr(self, std_col, self.data.index.values)
                elif file_col is None and std_col == 'description':
                    # Set empty descriptions if not available
                    setattr(self, std_col, np.array([''] * len(self.data)))
                elif file_col in self.data.columns:
                    setattr(self, std_col, self.data[file_col].values)
                else:
                    raise ValueError(f"Column '{file_col}' not found in the data file")
            
            # Calculate -log10(FDR) for volcano plot
            if self.log_fdr:
                self.log10_fdr = -np.log10(self.fdr)
            else:
                self.log10_fdr = self.fdr
            
            # Find indices of IDs to highlight
            if self.highlight_ids:
                self.set_highlight_ids(self.highlight_ids)
            
            return True
            
        except Exception as e:
            print(f"Error loading data: {e}")
            return False
    
    def set_highlight_ids(self, highlight_ids):
        """
        Set IDs to highlight and validate they exist in the data.
        
        Parameters:
        -----------
        highlight_ids : list
            List of IDs to highlight in the plots
            
        Returns:
        --------
        list
            List of IDs that were found in the data
        """
        self.highlight_ids = highlight_ids
        self.highlight_indices = []
        found_ids = []
        
        # Convert ids to strings for comparison (if they aren't already)
        id_strings = [str(id_val) for id_val in self.id]
        
        for h_id in highlight_ids:
            h_id_str = str(h_id)
            # Check if ID exists in the data
            if h_id_str in id_strings:
                idx = id_strings.index(h_id_str)
                self.highlight_indices.append(idx)
                found_ids.append(h_id)
            else:
                print(f"Warning: ID '{h_id}' not found in data")
        
        return found_ids
    
    def get_data_for_plotting(self):
        """
        Get all data in a format ready for plotting.
        
        Returns:
        --------
        dict
            Dictionary with all data needed for plotting
        """
        return {
            'log2fc': self.log2fc,
            'log10_fdr': self.log10_fdr,
            'avg_intensity': self.avg_intensity,
            'id': self.id,
            'description': self.description,
            'fdr': self.fdr,
            'highlight_indices': self.highlight_indices
        }

In [ ]:
#| export
# empty container for plotting data with standard names.
empty_data_dict = {
            'log2fc': None,
            'log10_fdr': None,
            'avg_intensity': None,
            'id': None,
            'description': None,
            'fdr': None,
            'highlight_indices': None
        }

In [ ]:
empty_data_dict

{'log2fc': None,
 'log10_fdr': None,
 'avg_intensity': None,
 'id': None,
 'description': None,
 'fdr': None,
 'highlight_indices': None}

## Visualization Function
> Now we'll define the function to generate the side-by-side volcano and MA plots:

In [ ]:
def create_volcano_ma_plots(plot_data, 
                          plot_title="Volcano and MA Plots", 
                          width=600, 
                          height=800):
    """
    Create one-on-top-of-the-other volcano and MA plots using Plotly.
    
    Parameters:
    -----------
    plot_data : PlotData
        A PlotData object containing the necessary data
    plot_title : str, optional
        Title for the plot (default: "Volcano and MA Plots")
    width : int, optional
        Width of the plot in pixels (default: 600)
    height : int, optional
        Height of the plot in pixels (default: 1000)
        
    Returns:
    --------
    plotly.graph_objects.Figure
        The created figure object
    """
    # Get the data for plotting
    plotting_data = plot_data.get_data_for_plotting()
    
    # Get highlighted indices
    highlight_indices = plotting_data['highlight_indices']
    
    # Create color array (black for normal, blue for highlighted)
    colors = ['black'] * len(plotting_data['log2fc'])
    for idx in highlight_indices:
        colors[idx] = 'blue'
    
    # Create size array (small for normal, larger for highlighted)
    sizes = [5] * len(plotting_data['log2fc'])
    for idx in highlight_indices:
        sizes[idx] = 10  # Make highlighted points larger
    
    # Create opacity array
    opacities = [0.5] * len(plotting_data['log2fc'])
    for idx in highlight_indices:
        opacities[idx] = 1.0  # Make highlighted points fully opaque
    
    # Create simplified hover text with only protein ID and description
    hover_texts = []
    for i in range(len(plotting_data['id'])):
        id_text = plotting_data['id'][i]
        desc_text = plotting_data['description'][i] if plotting_data['description'][i] else "No description"
        
        # Add a "HIGHLIGHTED" tag to highlighted points
        if i in highlight_indices:
            hover_texts.append(f"ID: {id_text}<br>Description: {desc_text}<br><b>HIGHLIGHTED</b>")
        else:
            hover_texts.append(f"ID: {id_text}<br>Description: {desc_text}")
    
    # Create figure with subplots - changed to 2 rows, 1 column
    fig = make_subplots(rows=2, cols=1,
                       subplot_titles=["Volcano Plot", "MA Plot"],
                       vertical_spacing=0.15)
    
    # Add Volcano plot (top)
    volcano_scatter = go.Scatter(
        x=plotting_data['log2fc'],
        y=plotting_data['log10_fdr'],
        mode='markers',
        marker=dict(
            color=colors,
            size=sizes,
            opacity=opacities,
            line=dict(width=1, color='black')
        ),
        text=hover_texts,
        hoverinfo='text',
        name='Volcano Plot',
        showlegend=False,
        hovertemplate='%{text}<extra></extra>'
    )
    
    fig.add_trace(volcano_scatter, row=1, col=1)
    
    # Add MA plot (bottom) - changed to row 2
    ma_scatter = go.Scatter(
        x=plotting_data['avg_intensity'],
        y=plotting_data['log2fc'],
        mode='markers',
        marker=dict(
            color=colors,
            size=sizes,
            opacity=opacities,
            line=dict(width=1, color='black')
        ),
        text=hover_texts,
        hoverinfo='text',
        name='MA Plot',
        showlegend=False,
        hovertemplate='%{text}<extra></extra>'
    )
    
    fig.add_trace(ma_scatter, row=2, col=1)
    
    # Update layout
    fig.update_layout(
        title=plot_title,
        width=width,
        height=height,
        showlegend=False,  # No legend
        hovermode='closest',
        # Make tooltip background transparent
        hoverlabel=dict(
            bgcolor="rgba(255, 255, 255, 0.1)",  # Semi-transparent white
            bordercolor="rgba(0, 0, 0, 0.3)",    # Semi-transparent border
            font=dict(color="black")             # Black text
        )
    )
    
    # Update x and y axis labels - fixed row/col references
    fig.update_xaxes(title_text="log2 Fold Change", row=1, col=1)
    fig.update_yaxes(title_text="-log10 FDR", row=1, col=1)
    
    fig.update_xaxes(title_text="Average Expression", row=2, col=1)
    fig.update_yaxes(title_text="log2 Fold Change", row=2, col=1)
    
    return fig

## Example Usage
> Let's demonstrate how to use these functions with a real dataset:

In [ ]:
column_mapping = {
    'log2fc': 'logFC',            # logFC column from your data
    'fdr': 'FDR',                # FDR column from your data
    'avg_intensity': 'log_AveExpr', # log_AveExpr column from your data
    'id': 'Gene_id',            # Gene_acc column from your data
    'description': 'Desc'         # Desc column from your data
}

file_path = '../tests/volcano_plots/for_web_limma_WT-C3.csv.zip'
# Create the PlotData instance
plot_data = PlotData(file_path, column_mapping, highlight_ids=['Blasticidin','Puromycin'])

# Print some information about the loaded data
print(f"Data loaded: {plot_data.data is not None}")
print(f"Number of rows: {len(plot_data.data) if plot_data.data is not None else 0}")
print(f"First few IDs: {plot_data.id[:5] if hasattr(plot_data, 'id') else 'Not loaded'}")
print(f"Highlight Indices: {plot_data.highlight_indices}")
# Quick access to all plotting data
plotting_data = plot_data.get_data_for_plotting()

Data loaded: True
Number of rows: 6205
First few IDs: ['Blasticidin' 'Puromycin' 'Tb05.5K5.100;Tb927.5.4450'
 'Tb05.5K5.110;Tb927.5.4460' 'Tb05.5K5.120;Tb927.5.4470']
Highlight Indices: [0, 1]


In [ ]:
#!rm iframe_figures/*

zsh:1: no matches found: iframe_figures/*


In [ ]:
fig = create_volcano_ma_plots(
    plot_data,
    plot_title="Differential Expression Analysis: Sample vs Control"
)
fig.show('iframe')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()